
markdown
Copy code
# 🧑‍🍳 **8 Week SQL Challenge: Case Study #1 - Danny's Diner**  
Welcome to my SQL learning journey! This project is part of the **8 Week SQL Challenge** created by **Danny Ma** to practice SQL skills through real-world case studies.  
<br>
<img src="https://8weeksqlchallenge.com/images/case-study-designs/1.png" width="500"/>
<br>
<br>

---

<br>

## 🍣 **Case Study Overview**  
Danny loves Japanese food and decided to open **Danny’s Diner**, a cozy restaurant serving his favorite dishes: **sushi, curry, and ramen**. To keep his business running smoothly, he needs help analyzing the restaurant’s performance and customer behavior using the captured data.

<br>

---

<br>

## 🧐 **Problem Statement**  
Danny wants insights on:  
1. **Customer visiting patterns**  
2. **Total spend by customers**  
3. **Popular menu items**  

These insights will help him personalize the customer experience and decide if he should expand the loyalty program.

<br>

---

<br>

## 📊 **Datasets Provided**  
Danny has shared three key datasets for analysis:  
- **sales**: Customer orders and dates  
- **menu**: Menu items and prices  
- **members**: Loyalty program members and join dates  

<br>

---

<br>

## 🎯 **Objective**  
The goal of this project is to write SQL queries to analyze the provided data, answer business questions, and generate actionable insights to support Danny’s decision-making.

👉 **Let's dive into the data and get started!**

<br>


---
<br>

🔗 **Reference:** [8 Week SQL Challenge - Case Study #1](https://8weeksqlchallenge.com/case-study-1/)


<br>


In [1]:
import sqlite3
import pandas as pd

In [2]:
# Create a connection to a SQLite database
conn = sqlite3.connect('dannys_diner.db')
cursor = conn.cursor()

# Create tables
cursor.executescript("""
CREATE TABLE sales (
  customer_id VARCHAR(1),
  order_date DATE,
  product_id INTEGER
);

INSERT INTO sales (customer_id, order_date, product_id) VALUES
  ('A', '2021-01-01', 1),
  ('A', '2021-01-01', 2),
  ('A', '2021-01-07', 2),
  ('A', '2021-01-10', 3),
  ('A', '2021-01-11', 3),
  ('B', '2021-01-01', 2),
  ('B', '2021-01-02', 2),
  ('B', '2021-01-04', 1),
  ('C', '2021-01-01', 3);

CREATE TABLE menu (
  product_id INTEGER,
  product_name VARCHAR(5),
  price INTEGER
);

INSERT INTO menu (product_id, product_name, price) VALUES
  (1, 'sushi', 10),
  (2, 'curry', 15),
  (3, 'ramen', 12);

CREATE TABLE members (
  customer_id VARCHAR(1),
  join_date DATE
);

INSERT INTO members (customer_id, join_date) VALUES
  ('A', '2021-01-07'),
  ('B', '2021-01-09');
""")
conn.commit()
print("Tables created successfully!")


Tables created successfully!


In [3]:
# Query the sales table
df_sales = pd.read_sql_query("SELECT * FROM sales", conn)
df_sales

,customer_id,order_date,product_id
0,A,2021-01-01,1
1,A,2021-01-01,2
2,A,2021-01-07,2
3,A,2021-01-10,3
4,A,2021-01-11,3
5,B,2021-01-01,2
6,B,2021-01-02,2
7,B,2021-01-04,1
8,C,2021-01-01,3


In [4]:
#Query the menu table
df_menu = pd.read_sql_query("SELECT * FROM menu", conn)
df_menu

,product_id,product_name,price
0,1,sushi,10
1,2,curry,15
2,3,ramen,12


In [5]:
#Query the members table
df_members = pd.read_sql_query("SELECT * FROM members", conn)
df_members

,customer_id,join_date
0,A,2021-01-07
1,B,2021-01-09


### **1. What is the total amount each customer spent at the restaurant?**

In [6]:
query = '''
SELECT
  s.customer_id,
  SUM(m.price) AS total_price
FROM sales AS s
JOIN menu AS m ON s.product_id = m.product_id
GROUP BY s.customer_id;
'''

df_amount_spent = pd.read_sql_query(query, conn)
df_amount_spent

,customer_id,total_price
0,A,64
1,B,40
2,C,12


### **2. How many days has each customer visited the restaurant?**

In [7]:
query = '''
SELECT *, COUNT (DISTINCT order_date) AS days_visited
FROM sales
GROUP BY customer_id
'''

df_customer_dates = pd.read_sql_query(query, conn)
df_customer_dates

,customer_id,order_date,product_id,days_visited
0,A,2021-01-01,1,4
1,B,2021-01-01,2,3
2,C,2021-01-01,3,1


### **3. What was the first item from the menu purchased by each customer?**

In [8]:
query = '''
SELECT *, MIN(product_name)
FROM sales AS s
JOIN menu AS m ON s.product_id = m.product_id
GROUP BY customer_id
'''

df_first_item = pd.read_sql_query(query,conn)
df_first_item

,customer_id,order_date,product_id,product_id,product_name,price,MIN(product_name)
0,A,2021-01-01,2,2,curry,15,curry
1,B,2021-01-01,2,2,curry,15,curry
2,C,2021-01-01,3,3,ramen,12,ramen


### **4. What is the most purchased item on the menu and how many times was it purchased by all customers?**

In [9]:
query = '''
SELECT product_name, COUNT (*) as count
FROM sales as s
JOIN menu as m ON s.product_id = m.product_id
GROUP BY product_name
'''

df_most_purchased = pd.read_sql_query(query,conn)
df_most_purchased

,product_name,count
0,curry,4
1,ramen,3
2,sushi,2


### **5. Which item was the most popular for each customer?**

In [10]:
query = '''
SELECT product_name, customer_id, COUNT (*) as count
FROM sales as s
JOIN menu as m ON s.product_id = m.product_id
GROUP BY customer_id
ORDER BY count DESC
'''

df_most_popular = pd.read_sql_query(query,conn)
df_most_popular

,product_name,customer_id,count
0,sushi,A,5
1,curry,B,3
2,ramen,C,1


### **6. Which item was purchased first by the customer after they became a member?**

In [21]:
query = """
SELECT s.customer_id, MIN(s.order_date) AS first_purchase_date, m.product_name
FROM sales s
JOIN members mem ON s.customer_id = mem.customer_id
JOIN menu m ON s.product_id = m.product_id
WHERE s.order_date >= mem.join_date
GROUP BY s.customer_id
"""
df_first_item= pd.read_sql(query, conn)
df_first_item

,customer_id,first_purchase_date,product_name
0,A,2021-01-07,curry


### **7. Which item was purchased just before the customer became a member?**

In [22]:
query = """
SELECT s.customer_id, MAX(s.order_date) AS last_purchase_date, m.product_name
FROM sales s
JOIN members mem ON s.customer_id = mem.customer_id
JOIN menu m ON s.product_id = m.product_id
WHERE s.order_date < mem.join_date
GROUP BY s.customer_id
"""
df_last_item = pd.read_sql(query, conn)
df_last_item

,customer_id,last_purchase_date,product_name
0,A,2021-01-01,sushi
1,B,2021-01-04,sushi


### **8. What is the total items and amount spent for each member before they became a member?**

In [23]:
query = """
SELECT s.customer_id, COUNT(s.product_id) AS total_items, SUM(menu.price) AS total_spent
FROM sales s
JOIN menu ON s.product_id = menu.product_id
JOIN members mem ON s.customer_id = mem.customer_id
WHERE s.order_date < mem.join_date
GROUP BY s.customer_id
"""
df_total_spend_before = pd.read_sql(query, conn)
df_total_spend_before

,customer_id,total_items,total_spent
0,A,2,25
1,B,3,40


### **9. If each $1 spent equates to 10 points and sushi has a 2x points multiplier - how many points would each customer have?**

In [24]:
query= """
SELECT s.customer_id,
       SUM(
           CASE
               WHEN menu.product_name = 'sushi' THEN menu.price * 10 * 2
               ELSE menu.price * 10
           END
       ) AS total_points
FROM sales s
JOIN menu ON s.product_id = menu.product_id
GROUP BY s.customer_id
"""
df_points = pd.read_sql(query, conn)
df_points

,customer_id,total_points
0,A,740
1,B,500
2,C,120


### **10. In the first week after a customer joins the program (including their join date) they earn 2x points on all items, not just sushi - how many points do customer A and B have at the end of January?**

In [25]:
query = """
SELECT s.customer_id,
       SUM(
           CASE
               WHEN s.order_date BETWEEN mem.join_date AND DATE(mem.join_date, '+6 days')
                    THEN menu.price * 10 * 2
               WHEN menu.product_name = 'sushi' THEN menu.price * 10 * 2
               ELSE menu.price * 10
           END
       ) AS total_points
FROM sales s
JOIN menu ON s.product_id = menu.product_id
LEFT JOIN members mem ON s.customer_id = mem.customer_id
GROUP BY s.customer_id
"""
df_points_jan = pd.read_sql(query, conn)
df_points_jan

,customer_id,total_points
0,A,1130
1,B,500
2,C,120
